In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import LambdaCallback
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt

In [ ]:
data = np.load('Wood_Training.npy', allow_pickle= True)
data2 = np.load('Wood_Testing.npy', allow_pickle= True)

data.shape, data2.shape

In [ ]:
def plot_random_spectral_curves(data, num_samples=2):
    
    # Randomly choose `num_samples` rows
    random_indices = np.random.choice(data.shape[0], num_samples, replace=False)

    # Set up the plot
    plt.figure(figsize=(10, 5))

    for i, idx in enumerate(random_indices):
        # Extract the spectral values (excluding the label)
        spectral_values = data[idx, :-1]

        # Extract the label (last column)
        label = data[idx, -1]

        # Create the x-axis for wavelength index (1 to 300)
        wavelengths = np.arange(1, spectral_values.shape[0] + 1)

        # Plot the spectral curve
        plt.plot(wavelengths, spectral_values, label=f"Sample {idx} - Label: {label}")


    plt.title(f"Spectral Curves for {num_samples} Random Samples")
    plt.xlabel("Bands")
    plt.ylabel("Mean Reflectance")
    plt.grid(True)
    plt.legend()
    plt.show()
    
plot_random_spectral_curves(data) 

In [ ]:
X_train = data[:, :-1]
y_train = data[:, -1]  

X_test = data2[:, :-1]
y_test = data2[:, -1]  

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)

scaler2 = StandardScaler()
scaler2.fit(X_test)

X_test = scaler2.transform(X_test)

In [ ]:
param_grid = {
    
    'n_neighbors': [3, 5],  # Use fewer values to test
    'weights': ['uniform'],  # Try with just one weight for now
    'metric': ['euclidean']  # Test one distance metric at a time
}

# KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search_knn = GridSearchCV(estimator=knn_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Training
grid_search_knn.fit(X_train, y_train)


In [ ]:
# test Accuracy
y_test_pred = grid_search_knn.predict(X_test)

accuracy = np.mean(y_test_pred == y_test) * 100
print("Test Accuracy:", accuracy)

In [ ]:
# Initialize the encoder
le= LabelEncoder()
y_train_encoded= le.fit_transform(y_train)

le2= LabelEncoder()
y_test_encoded= le2.fit_transform(y_test)

In [ ]:
unique_classes = np.unique(y_train)
print(unique_classes)

unique_classes_encoded = np.unique(y_train_encoded)
print(unique_classes_encoded)

In [ ]:
# Train XGBoost
xgb_classifier = XGBClassifier()


# Define parameter grid for GridSearchCV
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.01],
    'n_estimators': [100]
}


# GridSearchCV for XGBoost Classifier
xgb_grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=5, scoring='accuracy')
xgb_grid_search.fit(X_train, y_train_encoded)

# Get best parameters for XGBoost Classifier
best_params = xgb_grid_search.best_params_
print("Best Parameters:", best_params)

In [ ]:
# Get the best estimator
best_xgb_model = xgb_grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_xgb_model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test_encoded, y_pred)
print("Accuracy:", accuracy*100)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_dist = {
    
    'C': [0.1, 1, 10, 100],         
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4, 5], 
    'coef0': [0.0, 0.1, 0.5, 1.0] 
    
}
# SVM classifier
svc = SVC()

# Instantiate the grid search
random_search = RandomizedSearchCV(svc, param_distributions=param_dist, n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=-1)

# Training
random_search.fit(X_train, y_train)

In [ ]:

# Best parameters found by RandomizedSearchCV
print("Best parameters:", random_search.best_params_)

# Use the best model to predict on the test data
y_pred = random_search.best_estimator_.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100)


In [ ]:
from sklearn.naive_bayes import GaussianNB

# Initialize Gaussian Naive Bayes classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

In [ ]:
# test Accuracy
y_test_pred = nb_classifier.predict(X_test)

accuracy = np.mean(y_test_pred == y_test) * 100
print("Test Accuracy:", accuracy)

In [ ]:
config = tf.compat.v1.ConfigProto(gpu_options =
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
)

config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

In [ ]:
#Neural Network

tf.random.set_seed(1234) # for consistent results

model = Sequential(
    [
        tf.keras.Input(shape=(320,)),
        Dense(units=32, activation='relu', kernel_regularizer=regularizers.l2(0.01), name='layer1'),
        #Dense(units=16, activation='relu', kernel_regularizer=regularizers.l2(0.001), name='layer2'),
        #Dense(units=8, activation='relu', kernel_regularizer=regularizers.l2(0.0001), name='layer3'),
        
        Dense(units=2, activation='linear')
    ]
)

model.summary()

In [ ]:

#Training

model.compile(
    
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-2),
)

model.fit(
    
    X_train,y_train_encoded,
    batch_size = 500,
    epochs = 1000,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor = 'loss',
            patience = 500,
            restore_best_weights = True
        )
    ]
)

In [ ]:
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

#Accuracy
# Calculate total accuracy
total_accuracy = accuracy_score(y_test_encoded, predicted_labels)

print(f"Total Test Accuracy: {total_accuracy*100}")